## 定数定義
$$
V=\{1,\ldots,n\}:アイテムの集合\\
B=\{1,\ldots,n\}:ビンの集合\\
M=\{1,\ldots,m\}:グループの集合\\
w_i:アイテムiの大きさ\\
z_{il}\in\{0,1\}:アイテムiのグループlへの所属を表すバイナリ定数\\
r_l:グループlがビンを1つ使用するのにかかるコスト\\
C:ビンの容量\\
E=\{(s,t) ~|~ w_s+w_t > C~または~sとtは同じビンに割り当てられられない\}:枝の集合\\
G=(V,E):~VとEから成るグラフ\\
$$

## 変数定義
$$
x_{ik}\in\{0,1\}:アイテムiのビンkへの割り当てを表すバイナリ変数\\
y_k\in\{0,1\}:ビンkの使用を表すバイナリ変数\\
b_{kl}\in\{0,1\}:グループlのビンkの使用を表すバイナリ変数\\
$$

## 目的関数
$$
\begin{align}
\text{minimize} & \displaystyle\sum_{k=1}^n y_k + \sum_{k=1}^n \sum_{l=1}^m r_l b_{kl} - \sum_{l=1}^m r_l
\end{align}
$$

## 制約式
$$
\begin{align}
\text{subject to}
\displaystyle\sum_{i=1}^n w_i x_{ik} \leq C y_k&~~~~~(k=1,\dots,n)\\
\displaystyle\sum_{k=1}^n x_{ik} = 1&~~~~~(i=1,\dots,n)\\
x_{sk}+x_{tk} \leq 1&~~~~~((s,t)\in E,~k=1,\dots,n)\\
b_{kl} \geq x_{jk} z_{jl}&~~~~~(k=1,\dots,n,~l=1,\dots,m,~j=1,\dots,n)\\
x_{ik}\in\{0,1\}&~~~~~(i,k=1,\dots,n)\\
y_k\in\{0,1\}&~~~~~(k=1,\dots,n)\\
b_{kl}\in\{0,1\}&~~~~~(k=1,\dots,n,~l=1,\dots,m)\\
\end{align}
$$

# どういった手法にするか
- FFDなど
- メタヒューリスティクス使う **<--** いくつも作って性能の比較をする？ **<--** 各メタヒューリスティクスについて、参照されている論文を読む

# ファイルの読み込み 

In [1]:
filename = "/Users/yutaokamoto/Desktop/yutaokamoto_mac/Seminar/C125.9.clq.txt"
E=[]
with open(filename,mode="r") as f:
    lines = f.readlines()
    for i,v in enumerate(lines):
        line = v
        if "number of vertices" in line:
            n = int(line.split(" ")[-1])
        if line[0]=="e":
            start = i
            break
E = lines[start:-1] #ファイル名とEOFを除く
for i in range(len(E)):
    #E[i] = tuple(map(int,E[i][2:-1].split(" "))) #数字の数え方が１から始まってしまう
    temp = E[i][2:-1].split(" ")
    E[i] = (int(temp[0])-1,int(temp[1])-1)

# 問題設定

## 基本設定

In [2]:
import random
import numpy as np
random.seed(1)

#アイテムの個数
n

#アイテムの大きさ
w=[]
maximum=10
for i in range(n):
    w.append(random.randint(1,maximum))
    
#ビンの容量
C=maximum

#枝
E
    
#コンフリクト
conf={}
n_conf = len(E)
for i in range(n_conf):
    temp = [0]*n
    temp[E[i][0]-1] = 1
    temp[E[i][1]-1] = 1
    conf[f"conf{i}"] = temp
    

#グループ
G=[]#[tuple(np.arange(1,126))]

#グループを分けるコスト
r = [1]*len(G)

#ビンを使うコスト
bin_cost = 100

## グループ分け

In [3]:
N = np.arange(0,n) #アイテムのグループ分けの際にのみ使う定数
G  = []

random_list = list(np.random.choice(N,10))

index=0
while True:
    if random_list[index]>=len(N): #random[index]の値がNの個数より多かったらGにNをいれて終わり
        G.append(tuple(N))
        break
    group = random.sample(N.tolist(),random_list[index])
    N = np.array(list(set(N)-set(group)))
    G.append(tuple(group))
    index+=1

## コンフリクト辞書とノットコンフリクト辞書の作成

### コンフリクトがあるノード間に枝があるような辞書

In [4]:
N = np.arange(0,n) #辞書をつくる際にのみ使う定数
E_dict={}
notconf_E_dict={}
flag=E[0][0]
for i in range(len(E)):
    if flag!=E[i][0]:
        flag=E[i][0]
    E_dict[E[i][0]] = E_dict.setdefault(E[i][0],[]) + [E[i][1]]
    E_dict[E[i][1]] = E_dict.setdefault(E[i][1],[]) + [E[i][0]]
    notconf_E_dict[E[i][0]] = list(set(N)-set(E_dict[E[i][0]])-{E[i][0]})
    notconf_E_dict[E[i][1]] = list(set(N)-set(E_dict[E[i][1]])-{E[i][1]})

### コンフリクトがないノード間に枝があるような辞書

In [5]:
notconf_E = []
for i in notconf_E_dict:
    for j in notconf_E_dict[i]:
               notconf_E.append((i,j))

# FFD 

In [6]:
def capacity(w_i,bin_b,w,C): #bin_bの容量がw_iの大きさよりも大きいかを判定
    if (C-sum(w[i] for i in bin_b)) >= w_i:
        return True
    else:
        return False

In [7]:
def conflict(item,bin_b,E_dict): #bin_bに割り当てられているアイテムたちとw_iの間にコンフリクトがあるかを判定
    if item not in E_dict:
        return False
    for i in bin_b:
        if i in E_dict[item]:
                return False
    return True

In [8]:
def group(i,G): #アイテムiの属するグループの番号を返す関数
    for index,menber in enumerate(G):
        if i in menber:
            return index

In [9]:
def FFD(N,w,C,E_dict,G):
    bin_dict = {}
    bins_group_using = [-1]*len(G)
    for i in range(len(N)):
        bin_dict[i] = []
    for i in N: #アイテムをすべて走査
        for b in range(len(bin_dict)): #インデックスが小さい順にビンを走査
            if capacity(w[i],bin_dict[b],w,C) and conflict(i,bin_dict[b],E_dict): #容量制約を満たす　and　コンフリクトがない
                bin_dict[b] = bin_dict.setdefault(b,[]) + [i]
                bins_group_using[group(i,G)] += 1
                break  
    return bin_dict,bins_group_using

In [10]:
bin_dict,bins_group_using = FFD(N,w,C,E_dict,G)

In [11]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [0, 52]
1 [1]
2 [2, 21]
3 [3, 24]
4 [4, 19]
5 [5, 82]
6 [6, 80]
7 [7, 43]
8 [8, 25]
9 [9, 33]
10 [10, 14]
11 [11]
12 [12, 13]
13 [15]
14 [16, 29, 49]
15 [17]
16 [18, 112]
17 [20]
18 [22, 31]
19 [23, 77]
20 [26, 92]
21 [27, 30]
22 [28, 75]
23 [32, 106]
24 [34, 114]
25 [35, 74]
26 [36]
27 [37, 50]
28 [38]
29 [39]
30 [40, 48]
31 [41, 62]
32 [42, 64]
33 [44]
34 [45]
35 [46, 60]
36 [47, 54]
37 [51]
38 [53]
39 [55, 83]
40 [56]
41 [57]
42 [58]
43 [59]
44 [61]
45 [63]
46 [65, 89]
47 [66]
48 [67, 87]
49 [68, 93]
50 [69]
51 [70]
52 [71, 102]
53 [72]
54 [73]
55 [76]
56 [78]
57 [79]
58 [81]
59 [84]
60 [85]
61 [86]
62 [88]
63 [90]
64 [91, 96]
65 [94]
66 [95]
67 [97]
68 [98]
69 [99]
70 [100]
71 [101, 109]
72 [103]
73 [104]
74 [105]
75 [107]
76 [108]
77 [110]
78 [111]
79 [113]
80 [115]
81 [116]
82 [117]
83 [118]
84 [119]
85 [120]
86 [121]
87 [122]
88 [123]
89 [124]
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 []
110 [

In [12]:
print(bins_group_using)

[68, 17, 37]


In [13]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

90


# 走査するアイテムの順をシャッフルする

In [169]:
N = np.arange(0,n)
random.shuffle(N.tolist())

In [170]:
bin_dict,bins_group_using = FFD(N,w,C,E_dict,G)

In [171]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [0, 52]
1 [1]
2 [2, 21]
3 [3, 24]
4 [4, 19]
5 [5, 82]
6 [6, 80]
7 [7, 43]
8 [8, 25]
9 [9, 33]
10 [10, 14]
11 [11]
12 [12, 13]
13 [15]
14 [16, 29, 49]
15 [17]
16 [18, 112]
17 [20]
18 [22, 31]
19 [23, 77]
20 [26, 92]
21 [27, 30]
22 [28, 75]
23 [32, 106]
24 [34, 114]
25 [35, 74]
26 [36]
27 [37, 50]
28 [38]
29 [39]
30 [40, 48]
31 [41, 62]
32 [42, 64]
33 [44]
34 [45]
35 [46, 60]
36 [47, 54]
37 [51]
38 [53]
39 [55, 83]
40 [56]
41 [57]
42 [58]
43 [59]
44 [61]
45 [63]
46 [65, 89]
47 [66]
48 [67, 87]
49 [68, 93]
50 [69]
51 [70]
52 [71, 102]
53 [72]
54 [73]
55 [76]
56 [78]
57 [79]
58 [81]
59 [84]
60 [85]
61 [86]
62 [88]
63 [90]
64 [91, 96]
65 [94]
66 [95]
67 [97]
68 [98]
69 [99]
70 [100]
71 [101, 109]
72 [103]
73 [104]
74 [105]
75 [107]
76 [108]
77 [110]
78 [111]
79 [113]
80 [115]
81 [116]
82 [117]
83 [118]
84 [119]
85 [120]
86 [121]
87 [122]
88 [123]
89 [124]
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 []
110 [

In [172]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

90


# コンフリクトの数が多い順に走査

In [173]:
N = np.arange(0,n)
N = sorted(E_dict,key=lambda x:len(E_dict[x]),reverse=True)

In [174]:
N

[53,
 59,
 113,
 44,
 103,
 110,
 6,
 7,
 18,
 79,
 98,
 100,
 109,
 10,
 28,
 39,
 48,
 66,
 97,
 124,
 17,
 23,
 38,
 46,
 69,
 9,
 21,
 33,
 58,
 64,
 76,
 81,
 91,
 122,
 1,
 5,
 8,
 24,
 29,
 40,
 45,
 47,
 51,
 73,
 84,
 85,
 95,
 118,
 0,
 3,
 4,
 12,
 30,
 34,
 43,
 56,
 57,
 68,
 71,
 78,
 90,
 108,
 114,
 116,
 117,
 19,
 22,
 25,
 26,
 36,
 42,
 52,
 61,
 65,
 83,
 105,
 106,
 120,
 121,
 27,
 31,
 55,
 60,
 70,
 86,
 92,
 119,
 123,
 13,
 20,
 37,
 54,
 62,
 77,
 88,
 101,
 104,
 111,
 11,
 16,
 32,
 49,
 67,
 80,
 99,
 102,
 115,
 2,
 14,
 15,
 41,
 63,
 87,
 93,
 94,
 96,
 112,
 50,
 72,
 89,
 74,
 75,
 107,
 35,
 82]

In [175]:
#降順になっているか確かめ
len(E_dict[53])

119

In [176]:
bin_dict,bins_group_using = FFD(N,w,C,E_dict,G)

In [177]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [53]
1 [59, 16]
2 [113, 23]
3 [44, 92]
4 [103, 37]
5 [110]
6 [6, 80]
7 [7, 43]
8 [18, 19]
9 [79, 2]
10 [98]
11 [100]
12 [109, 101]
13 [10, 58]
14 [28, 75]
15 [39, 74]
16 [48, 40]
17 [66, 0]
18 [97, 25]
19 [124, 82]
20 [17]
21 [38]
22 [46, 81]
23 [69, 106]
24 [9, 33]
25 [21, 3]
26 [64, 12]
27 [76, 62]
28 [91, 42]
29 [122, 71]
30 [1]
31 [5]
32 [8, 31]
33 [24, 34]
34 [29, 70]
35 [45]
36 [47, 56]
37 [51]
38 [73]
39 [84]
40 [85]
41 [95, 114]
42 [118]
43 [4, 22]
44 [30, 27]
45 [57]
46 [68, 93]
47 [78, 49]
48 [90]
49 [108]
50 [116, 96]
51 [117]
52 [26]
53 [36]
54 [52]
55 [61]
56 [65, 89]
57 [83, 55]
58 [105]
59 [120]
60 [121]
61 [60]
62 [86]
63 [119, 54]
64 [123]
65 [13, 88]
66 [20]
67 [77, 67]
68 [104]
69 [111]
70 [11]
71 [32]
72 [99]
73 [102]
74 [115]
75 [14]
76 [15]
77 [41]
78 [63]
79 [87]
80 [94]
81 [112, 50]
82 [72]
83 [107]
84 [35]
85 []
86 []
87 []
88 []
89 []
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
10

In [178]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

85


# 大きさあたりのコンフリクトの数が多い順に走査

In [179]:
N = np.arange(0,n)
N = sorted(E_dict,key=lambda x:len(E_dict[x])/w[x],reverse=True)

In [180]:
bin_dict,bins_group_using = FFD(N,w,C,E_dict,G)

In [181]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [23, 82]
1 [24, 27, 37]
2 [12, 47]
3 [43, 89, 96]
4 [114, 62, 41]
5 [25, 10]
6 [106, 49, 22]
7 [31, 74, 33]
8 [92, 3]
9 [16, 71]
10 [80, 67]
11 [46, 83]
12 [21, 4]
13 [2, 88]
14 [109, 75]
15 [0, 55]
16 [39]
17 [9, 19]
18 [64, 42]
19 [91, 119]
20 [29, 112]
21 [40, 54]
22 [93, 68]
23 [103]
24 [18]
25 [48, 56]
26 [38]
27 [78]
28 [116]
29 [70]
30 [123]
31 [101]
32 [99]
33 [50]
34 [113]
35 [44]
36 [28]
37 [66]
38 [97]
39 [124]
40 [8]
41 [30]
42 [52]
43 [65]
44 [121]
45 [60]
46 [13]
47 [77]
48 [14]
49 [87]
50 [107]
51 [6]
52 [7]
53 [79]
54 [17]
55 [58]
56 [81]
57 [122]
58 [5]
59 [34]
60 [11]
61 [102]
62 [115]
63 [63]
64 [53]
65 [59]
66 [72]
67 [110]
68 [98]
69 [69]
70 [35]
71 [76]
72 [45]
73 [51]
74 [73]
75 [95]
76 [118]
77 [90]
78 [108]
79 [26]
80 [36]
81 [120]
82 [104]
83 [111]
84 [32]
85 [94]
86 [100]
87 [1]
88 [84]
89 [85]
90 [57]
91 [117]
92 [61]
93 [105]
94 [86]
95 [20]
96 [15]
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 []
110 []
111 []
112 [

In [182]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

97


# 大きさあたりのコンフリクトの数が少ない順に走査

In [183]:
N = np.arange(0,n)
N = sorted(E_dict,key=lambda x:len(E_dict[x])/w[x],reverse=False)

In [184]:
bin_dict,bins_group_using = FFD(N,w,C,E_dict,G)

In [185]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [15]
1 [20]
2 [86]
3 [61]
4 [105]
5 [57]
6 [117]
7 [1]
8 [84]
9 [85]
10 [100]
11 [94, 62]
12 [32, 16]
13 [104]
14 [111, 80]
15 [26, 92]
16 [36, 25]
17 [120, 12]
18 [90, 82]
19 [108, 43]
20 [45]
21 [51]
22 [73]
23 [95, 27]
24 [118, 114]
25 [76, 106]
26 [35, 74]
27 [69]
28 [98]
29 [110, 23]
30 [72]
31 [53]
32 [59]
33 [41, 2]
34 [63, 24]
35 [11]
36 [102, 71]
37 [115]
38 [34]
39 [5]
40 [58, 10]
41 [81, 49]
42 [122]
43 [17]
44 [6]
45 [7]
46 [79]
47 [107, 0]
48 [14, 31]
49 [87, 67]
50 [13, 88]
51 [77, 89]
52 [60, 21]
53 [52, 75]
54 [65]
55 [121]
56 [30]
57 [8]
58 [28]
59 [66]
60 [97]
61 [124]
62 [44]
63 [113]
64 [50, 112]
65 [99, 54]
66 [101, 119]
67 [70, 29]
68 [123, 40]
69 [22, 4]
70 [68, 93]
71 [78]
72 [116, 96]
73 [38]
74 [55, 83]
75 [42, 64]
76 [3, 33]
77 [56, 48]
78 [18, 19]
79 [103, 37]
80 [9]
81 [91]
82 [39]
83 [47]
84 [109]
85 [46]
86 []
87 []
88 []
89 []
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 

In [186]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

86


# コンフリクトあたりの大きさが大きい順に走査

In [195]:
N = np.arange(0,n)
N = sorted(E_dict,key=lambda x:w[x]/len(E_dict[x]),reverse=True)

In [196]:
bin_dict,bins_group_using= FFD(N,w,C,E_dict,G)

In [197]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [15]
1 [20]
2 [86]
3 [61]
4 [105]
5 [57]
6 [117]
7 [1]
8 [84]
9 [85]
10 [100]
11 [94, 62]
12 [32, 16]
13 [104]
14 [111, 80]
15 [26, 92]
16 [36, 25]
17 [120, 12]
18 [90, 82]
19 [108, 43]
20 [45]
21 [51]
22 [73]
23 [95, 27]
24 [118, 114]
25 [76, 106]
26 [35, 74]
27 [69]
28 [98]
29 [110, 23]
30 [72]
31 [53]
32 [59]
33 [41, 2]
34 [63, 24]
35 [11]
36 [102, 71]
37 [115]
38 [34]
39 [5]
40 [58, 10]
41 [81, 49]
42 [122]
43 [17]
44 [6]
45 [7]
46 [79]
47 [107, 0]
48 [14, 31]
49 [87, 67]
50 [13, 88]
51 [77, 89]
52 [60, 21]
53 [52, 75]
54 [65]
55 [121]
56 [30]
57 [8]
58 [28]
59 [66]
60 [97]
61 [124]
62 [44]
63 [113]
64 [50, 112]
65 [99, 54]
66 [101, 119]
67 [70, 29]
68 [123, 40]
69 [22, 4]
70 [68, 93]
71 [78]
72 [116, 96]
73 [38]
74 [55, 83]
75 [42, 64]
76 [3, 33]
77 [56, 48]
78 [18, 19]
79 [103, 37]
80 [9]
81 [91]
82 [39]
83 [47]
84 [109]
85 [46]
86 []
87 []
88 []
89 []
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 

In [198]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

86


# コンフリクトあたりの大きさが小さい順に走査

In [199]:
N = np.arange(0,n)
N = sorted(E_dict,key=lambda x:w[x]/len(E_dict[x]),reverse=False)

In [200]:
bin_dict,bins_group_using= FFD(N,w,C,E_dict,G)

In [201]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [23, 82]
1 [24, 27, 37]
2 [12, 47]
3 [43, 89, 96]
4 [114, 62, 41]
5 [25, 10]
6 [106, 49, 22]
7 [31, 74, 33]
8 [92, 3]
9 [16, 71]
10 [80, 67]
11 [46, 83]
12 [21, 4]
13 [2, 88]
14 [109, 75]
15 [0, 55]
16 [39]
17 [9, 19]
18 [64, 42]
19 [91, 119]
20 [29, 112]
21 [40, 54]
22 [93, 68]
23 [103]
24 [18]
25 [48, 56]
26 [38]
27 [78]
28 [116]
29 [70]
30 [123]
31 [101]
32 [99]
33 [50]
34 [113]
35 [44]
36 [28]
37 [66]
38 [97]
39 [124]
40 [8]
41 [30]
42 [52]
43 [65]
44 [121]
45 [60]
46 [13]
47 [77]
48 [14]
49 [87]
50 [107]
51 [6]
52 [7]
53 [79]
54 [17]
55 [58]
56 [81]
57 [122]
58 [5]
59 [34]
60 [11]
61 [102]
62 [115]
63 [63]
64 [53]
65 [59]
66 [72]
67 [110]
68 [98]
69 [69]
70 [35]
71 [76]
72 [45]
73 [51]
74 [73]
75 [95]
76 [118]
77 [90]
78 [108]
79 [26]
80 [36]
81 [120]
82 [104]
83 [111]
84 [32]
85 [94]
86 [100]
87 [1]
88 [84]
89 [85]
90 [57]
91 [117]
92 [61]
93 [105]
94 [86]
95 [20]
96 [15]
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 []
108 []
109 []
110 []
111 []
112 [

In [202]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

97
